# An example of neutrino propagation simulation through flexOPT.jl 
Some of functions are the fruit of works from Estelle Salomé (July-August 2025) and TIPE (2025-2026).

i need to make a lazy option (not only regarding DrWatson)

In [1]:
using Pkg


cd(@__DIR__)
Pkg.activate("../")
ParamFile = "../config/testparam.csv" # 1D Earth/Mars/Moon models are defined

include("../src/commonBatchs.jl")
include("../src/planet1D.jl")
include("../src/GeoPoints.jl")

include("../src/Neurthino.jl")

using .commonBatchs, .planet1D, .GeoPoints
using .Neurthino 
using Colors



  Activating project at `~/Documents/Github/flexOPT`


geodynamical model directory (Takashi \& Paul, we cannot share the models online!)
Paul is ready to give us more models whenever we want but the data are confidential and I cannot share on the Github

By calling .Neurthino, the user already chose :NormalThree as DEFAULT_NEUTRINO_FLAVOURS[], with decimated U and H, 
but if you want to use not decimated value for U and H or you want to propose any other flavour configuration, do this!

`U, H=set_default_flavour!(:NormalThree;roundingU=0.e0,roundingH=0.e0) `

In [ ]:
dir="/Users/nobuaki/Documents/MantleConvectionTakashi/op_old_full_mars_2025/"
rhoFiles=myListDir(dir; pattern=r"test_rho\d");
compositionFiles=myListDir(dir; pattern=r"test_c\d");
temperatureFiles=myListDir(dir; pattern=r"test_t\d");
wtrFiles=myListDir(dir; pattern=r"test_wtr\d");

## making 2D(/3D) box with the aid of GeoPoint.jl, reading 1D Earth model (cf. ParamFile) using planet1D.jl

Please refer to getRegionalBox.ipynb if you have some questions 

In [ ]:
set_default_planet!(:SphericalEarth) # the default is ":Earth"
#SphericalEarth means a perfect sphere with a radius of 6371000.0 m 
# otherwise of course, I made an Earth with ellipticity 

#planet1D.my1DDSMmodel=getSet1Dmodel!(:ak135) # if you want to try other 1D models, you can call getSet1Dmodel!(:name; modelPath="model_0000.nd")

#planet1D.my1DDSMmodel=getSet1Dmodel!(:PREM) 

In [ ]:
# you can do some more exercises with getRegionalBox.ipynb but just trust me here

# two (extreme) points that can define the slice (or the x-y local plane for 3D box)
p1 = GeoPoint(0.0,0.0;alt=30.e3) # lat, lon in degrees, altitude in metre (as an option)
# if you call GeoPoints with three floats without ; for alt, it will interpret as x, y, z in cartesian ecef coordinates
p2 = GeoPoint(0.0,180.0;alt=30.e3) # 


Δx = 25.e3 # in metre
Δz = 25.e3

altMax = 6500.e3
altMin = -6500.e3

as you can see in the GeoPoints.jl example, we can construct 3D box too, which can be maybe a good idea to explore the uncertainties of azimuth of incident angle.

In [ ]:
boxGrids=constructLocalBox(p1,p2,Δx,Δz,altMin,altMax;centreOption ="centreOfPlanet")

In [ ]:
# getting model parameters on the 2D disk
seismicModel=lazyProduceOrLoad("seismicModel2DGlobal",getParamsWithoutTopo,boxGrids.allGridsInGeoPoints,boxGrids.effectiveRadii)

In [ ]:
using CairoMakie
xvals = [p.xz[1] for p in boxGrids.allGridsInCartesian[:,1]]*1.e-3
zvals = [p.xz[2] for p in boxGrids.allGridsInCartesian[1,:]]*1.e-3
fig, ax, hm = heatmap(
    #topo.x,topo.y,topo.z';
    #collect((0:1:(Nx-1)).*Δx).*1.e-3,(collect(0:1:(Nz-1)).*Δz.+altMin).*1.e-3, seismicModel.ρ;
    xvals, zvals, seismicModel.ρ ;
    colormap = :inferno,
    #colorrange=(0,4),
    axis = (aspect = DataAspect(), xlabel = "horizontal", ylabel = "altitude from p1", title = "Density model")
)
Colorbar(fig[1,2], hm, label="density")
fig

# Superposition or Replacement by tomographic/geodynamic 2D/3D models

Tell me if you have your own format of the model, I will try to make buffer functions

In [ ]:
function readStagYYFile(file,xvals,yvals;getAverage=true)
    field, avField, diffField, Xnode, Ynode, rcmb, boolFlat = readStagYYFile(file)
    #extendToCoreWithρ!(field, Xnode, Ynode, rcmb, dR, iCheckCoreModel=false)
    #quarterDiskExtrapolationRawGrid!(field, Xnode, Ynode)
    zvals = 
end

In [ ]:
function readStagYYFile(file,xvals,yvals,zvals;getAverage=true)
    field, avField, diffField, Xnode, Ynode, rcmb, boolFlat = readStagYYFile(file)
    #extendToCoreWithρ!(field, Xnode, Ynode, rcmb, dR, iCheckCoreModel=false)
    #quarterDiskExtrapolationRawGrid!(field, Xnode, Ynode)

    correlationLength=(xvals[2]-xvals[1], yvals[2]-yvals[1],zvals[2]-zvals[1]) # not yet fully understood this for DIV interpolation
    epsilon2 =1.0 # not yet fully understood this for DIV interpolation
    field, avNewField, diffNewField, Xnode, Ynode, rcmb, boolFlat = readStagYYFile(file)


    if boolFlat
        #nZ=1
        minZ=0.0
        maxZ=0.0
        tmpX=correlationLength[1]
        tmpY=correlationLength[2]
        correlationLength=(tmpX,tmpY)

        mask,(pm,pn),(xi,yi) = DIVAnd_rectdom(range(minX,stop=maxX,length=nX),
                                                range(minY,stop=maxY,length=nY));
    else

        mask,(pm,pn,po),(xi,yi,zi) = DIVAnd_rectdom(range(minX,stop=maxX,length=nX),
                                                range(minY,stop=maxY,length=nY),
                                                range(minZ,stop=maxZ,length=nZ));
    end


    fiCar,_ = DIVAndrun(mask,(pm,pn),(xi,yi),(Xnode,Ynode),field,correlationLength,epsilon2);
    avFiCar,_ = DIVAndrun(mask,(pm,pn),(xi,yi),(Xnode,Ynode),field,correlationLength,epsilon2);
    diffFiCar,_ = DIVAndrun(mask,(pm,pn),(xi,yi),(Xnode,Ynode),field,correlationLength,epsilon2);

    return fiCar, avFiCar, diffFiCar
end

In [ ]:

function readStagYYFile(file;getAverage=true)
    
    # new StagYY file reading function

    # for instance I did not put getAverage=false option


    magic, inputILEN, byte_reverse_in, io=read_magic(file)
    magic,nval = evaluate_nval_from_magicNumber(magic)
    intDataType,floatDataType,dx,dy,nx,ny,nz,nb,nnx,nny,nnz,nnb,rcmb,iStep,time,zc,boolSpherical=read_header(io,inputILEN,magic)
   
    nodes=nothing
    boolFlat = true

    Xnode=[]
    Ynode=[]    
    Znode=[]

    if nx == 1
       boolFlat = true
    end
 
    if boolSpherical
        # these are the grid points including the boundaries
        r = rcmb .+ zc[1,1:nz+1]
        
        theta = (collect(1:1:nx+1) .- (nx+1)) .* dx .+ 0.5*π
        phi = (collect(1:1:ny+1) .- (ny+1)) .* dy
        

        # these are the midpoints 
        r_mid = rcmb .+ zc[2,1:nz]
        theta_mid = (collect(1:1:nx) .- (nx+1)) .* dx .+ 0.5*π .+ 0.5*dx
        phi_mid = phi = (collect(1:1:ny) .- (ny+1)) .* dy .+ 0.5*dy
        
        # interpolation should be done with the midpoints + end points

        r_new = prepend!(r_mid,r[1])
        r_new = push!(r_mid,r[end])
        minR = r[1]
        maxR = r[end]
        rC=r_new

        theta_new = prepend!(theta_mid,theta[1])
        theta_new = push!(theta_new,theta[end])
        minθ = theta[1]
        maxθ = theta[end]
        phi_new = prepend!(phi_mid,phi[1])
        phi_new = push!(phi_new,phi[end])
        minϕ = phi[1]
        maxϕ = phi[end]
        if boolFlat
            minθ = 0.
            maxθ = 0.
            theta_new = (π/2)  # if flat, we are looking at the 
            nodes=(theta_new,phi_new,r_new)
        else   
            nodes=(theta_new,phi_new,r_new)
        end

    
    else
        # do not know if this works (certainly not! try to mimic the code above for the spherical case)
        x=collect(1:1:nx+1) .* dx
        y=collect(1:1:ny+1) .* dy
        z=zc
        nodes=(x,y,z)
    end

    

    theta_new_piCoef = theta_new ./ π
    phi_new_piCoef = phi_new ./ π

    if boolSpherical
        for rTemp in r_new
            for phiTemp in phi_new_piCoef
                cosPhiPi = cospi(phiTemp)
                sinPhiPi = sinpi(phiTemp)
               
                for thetaTemp in theta_new_piCoef
                    cosThetaPi = cospi(thetaTemp)
                    sinThetaPi = sinpi(thetaTemp) 
                    Xnode = push!(Xnode,rTemp*sinThetaPi*cosPhiPi)
                    Ynode = push!(Ynode,rTemp*sinThetaPi*sinPhiPi)
                    Znode = push!(Znode,rTemp*cosThetaPi)

                end
            end
        end
    else
        @error "cartesian format: not yet developed"
    end


   rawField = readField(io,floatDataType,nval,nx,ny,nz,nb,nnx,nny,nnz,nnb)

   
    if boolFlat
        #field=reshape(field, (ny,nz)) 

        field = zeros(floatDataType,ny,nz)
        field[1:ny,1:nz]=rawField[1,1,1:ny,1:nz,1]


        newField = zeros(floatDataType,ny+2,nz+2)

        # the interior

        newField[2:ny+1,2:nz+1] = field[1:end,1:end]

        # 4 'surfaces'

        newField[1,2:nz+1] = field[1,1:nz]
        newField[end,2:nz+1] = field[end,1:nz]
        newField[2:ny+1,1] = field[1:ny,1]
        newField[2:ny+1,end] = field[1:ny,end]

        # 4 'endpoints'

        newField[1,1,1]=field[1,1,1]
        newField[1,end,1]=field[1,end,1]
        newField[1,1,end]=field[1,1,end]
        newField[1,end,end]=field[1,end,end]


    else
        #field=reshape(field, (nx,ny,nz)) 
        field = zeros(floatDataType,nx,ny,nz)
        field[1:ny,1:nz]=rawField[1,1:nx,1:ny,1:nz,1]

        newField = zeros(floatDataType,nx+2,ny+2,nz+2)

        # the interior

        newField[2:nx+1,2:ny+1,2:nz+1] = field[1:nx,1:ny,1:nz]

        # 6 'surfaces'
        
        newField[1,2:ny+1,2:nz+1] = field[1,1:ny,1:nz]
        newField[end,2:ny+1,2:nz+1] = field[end,1:ny,1:nz]
        newField[2:nx+1,1,2:nz+1] = field[1:nx,1,1:nz]
        newField[2:nx+1,end,2:nz+1] = field[1:nx,end,1:nz]
        newField[2:nx+1,2:ny+1,1] = field[1:nx,1:ny,1]
        newField[2:nx+1,2:ny+1,end] = field[1:nx,1:ny,end]

        # 8 'endpoints'
        newField[1,1,1]=field[1,1,1]
        newField[end,1,1]=field[end,1,1]
        newField[1,end,1]=field[1,end,1]
        newField[end,end,1]=field[end,end,1]
        newField[1,1,end]=field[1,1,end]
        newField[end,1,end]=field[end,1,end]
        newField[1,end,end]=field[1,end,end]
        newField[end,end,end]=field[end,end,end]

    end

    if boolFlat
        avNewField = similar(newField)
        diffNewField = similar(newField)
        for iz in 1:nz+2
            average = sum(newField[:,iz])
            average /= Float64(ny+2)
            avNewField[:,iz] .= average
        end
        #@show size(avNewField)
        diffNewField = newField .- avNewField
        newField=reshape(newField,(ny+2)*(nz+2))
        avNewField=reshape(avNewField,(ny+2)*(nz+2))
        diffNewField=reshape(diffNewField,(ny+2)*(nz+2))
        return newField, avNewField, diffNewField, Xnode, Ynode, rcmb, boolFlat
    else
        for iz in 1:nz+2
            average = sum(newField[:,iz])
            average /= Float64(ny+2)
            avNewField[:,iz] .= average
        end
        #@show size(avNewField)
        diffNewField = newField .- avNewField
        newField=reshape(newField,(ny+2)*(nz+2))
        avNewField=reshape(avNewField,(ny+2)*(nz+2))
        diffNewField=reshape(diffNewField,(ny+2)*(nz+2))
        #newField=reshape(newField,(nx+2)*(ny+2)*(nz+2))
        return newField, avNewField, diffNewField, Xnode, Ynode, rcmb, boolFlat
        
    end
   #fieldInterpolated=interpolate(nodes,newField,Gridded(Linear()))

end



In [ ]:
readStagYYFile